In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import utils
from tensorflow.keras import *
from tensorflow.keras.layers import *
import glob
import numpy as np
from numpy.random import choice
from random import shuffle, seed
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Nadam
import os.path
import pickle
from music21 import *

In [0]:
if (not os.path.exists('maestro-v2.0.0-midi.zip')):
    !wget https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip

if (not os.path.exists('maestro-v2.0.0')):
    !unzip maestro-v2.0.0-midi.zip

In [4]:
def get_list_midi(filepath = 'C:/Users/ongjx/Documents/GitHub/ML-Music/maestro-v2.0.0/**/*.midi', rand_seed = 999):
    midi_list = glob.glob(filepath)
    seed(rand_seed)
    shuffle(midi_list)
    return midi_list

midi_list = get_list_midi()

In [5]:
len(midi_list)

1282

## Run this cell below only if you want to generate notes from new midi files

In [7]:
def get_notes_parallel(file):
    notes = []

    midi = converter.parse(file)

    print("Parsing %s" % file)

    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    print(parts)
    try:   
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse()
    except:  
        notes_to_parse = midi.flat.notes
        
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
    
    return notes

notes = common.runParallel(midi_list[0:25], parallelFunction=get_notes_parallel, updateFunction=True)

In [8]:
notes = common.runParallel(midi_list[0:25], parallelFunction=get_notes_parallel, updateFunction=True)

Done 0 tasks of 25
Done 21 tasks of 25
Done 25 tasks of 25


In [25]:
with open(r'data/notes', 'wb') as filepath:
    pickle.dump(notes, filepath)

In [10]:
notes = [item for list in notes for item in list]

n_vocab = len(set(notes))
pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

## Run this cell below if you have already created a pickle file for notes

In [28]:
objects = []
with (open(r"C:\Users\ongjx\Documents\GitHub\ML-Music\data\notes", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break
            
notes = objects[0]
n_vocab = len(set(notes))
pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

## Model preparation and setup

In [13]:
sequence_length = 150
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = utils.to_categorical(network_output)

In [14]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]


In [0]:
 model.fit(network_input, network_output, epochs=10, batch_size=64, callbacks=callbacks_list)

W0905 05:57:40.332987 139849103964032 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
 34944/118264 [=======>......................] - ETA: 3:07:44 - loss: 4.8614